In [ ]:
# Import packages
import TS_Custom
from TS_Custom import *

    

#import FIXEDSTrategy_Multithread_YFData
#from FIXEDSTrategy_Multithread_YFData import *

sellresults_filename = "zlogs_alphaedge_sellresults_df" + ".csv"

    
def getStrategyResult(position):
    #for symbol in symbol_list :
    
    try:
        
        
        #print (datetime.now().strftime('%H:%M'))

        
        # get options data from td ameritrade
        #content = getOptionChainDatafromTD(symbol)

        # retrieve call symbol
        #call_optionSymbol = getCallOptionSymbol(content)

        # retrieve put symbol
        #put_optionSymbol = getPutOptionSymbol(content)   

                 
        #print to console
        #printToConsole(df)
        

        # Strategy Code
        
        symbol = position['SYMBOL']
        positionType = position['POSITION_TYPE']
        quantity =position['QUANTITY']
        avgPrice = position['AVGPRICE']
        bidPrice = position['BID']
        askPrice = position['ASK']
        openpl = position['OPENPL']
        openpl_percent = position['OPENPL_PERCENT']
        triggerprice = position['TRIGGER_PRICE']
        previous_openpl = position['PREVIOUS_OPENPL']
        profit_target = position['PROFIT_TARGET']
        stop_loss = position['STOP_LOSS']
        #limitprice = round((askPrice + bidPrice)/2,2)
        limitprice = round((bidPrice),2)
        
        val = symbol.split(" ")
        
        # get price history from tradestation
        barsBack = 75
        minutes = 1
        if (len(val) > 1):
            df = getPriceHistory_TS(val[0],str(barsBack), str(minutes))
        else :
            df = getPriceHistory_TS(symbol,str(barsBack), str(minutes))
                   
        #if (df != []):
        
        
        
        
        #convert current candles to heiken-ashi
        df = getHA(df)
        #print(df.iloc[[98]])        

        ema9 = df['ema9']        
        
        df = Supertrend(df, 10, 1)
        
        
        #retrieve keys
        last_key = ema9.keys()[-1]
        previous_key = ema9.keys()[-2]
        previous_key_2 = ema9.keys()[-3]
        previous_key_3 = ema9.keys()[-4]
        

        #variables
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        strategy = 'SELL_SUPERTREND_MTF'

        selloptions = False
        
        # if the position is an option then sell according to the rules below
        if (len(val) > 1) :
            if (selloptions):                    

                if (val[1].__contains__("C")):
                    #if (openpl_percent > 2) or (openpl_percent < -8) :
                    if ((df['HA_Close'][previous_key] < df['HA_Open'][previous_key]) and (df['HA_Open'][previous_key] == df['HA_High'][previous_key])) :
                        print (datetime.now().strftime('%H:%M:%S'))
                        print('Sell Call for ', symbol, 'at', limitprice)
                        profitloss = round((quantity * 100 * (limitprice-avgPrice)),2)

                        # Sell Call in TS
                        #sellCALL(symbol, quantity, df,val, tradelog_filename, content)    


                        #add to DB
                        addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_CALL', limitprice, quantity)
                        addprofitlossOutputtoDB(profitloss)
                        deletePositionfromDB(symbol)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_CALL', limitprice, quantity, profitloss, bidPrice, askPrice,df)



                if (val[1].__contains__("P")):

                    #if (openpl_percent > 2) or (openpl_percent < -8) :
                    if ((df['HA_Close'][previous_key] > df['HA_Open'][previous_key]) and (df['HA_Open'][previous_key] == df['HA_Low'][previous_key])) :
                        print (datetime.now().strftime('%H:%M:%S'))
                        print('Sell Put for ', symbol, 'at', limitprice)
                        profitloss = round((quantity * 100 * (limitprice-avgPrice)),2)

                        #Sell Put in TS
                        #sellPUT(symbol, quantity, df, val, tradelog_filename, content)    


                        #add to DB
                        addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_PUT', limitprice, quantity)
                        addprofitlossOutputtoDB(profitloss)
                        deletePositionfromDB(symbol)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_PUT', limitprice, quantity, profitloss, bidPrice, askPrice,df)


        # if the position is a stock then sell according to the rules below
        else :

            if (positionType == 'Long'):
                
                #(df['ST_Upperband'][previous_key]> 0) or 
                if ((bidPrice <= stop_loss) or (bidPrice >= profit_target)):


                    #if (openpl_percent >= 0.03 or openpl_percent <= -0.30) :
                    #if (openpl_percent >= 0.1) :

                    print(df[['SYMBOL','TimeStamp','Close','Open','High','Low','K_slow','D_slow','ST_Upperband','ST_Lowerband']].tail(5))

                    print (datetime.now().strftime('%H:%M:%S'))
                    limitprice = round((bidPrice-0.01),2)
                    print('Sell Stock for ', symbol, 'at', limitprice, 'with p/l = $', openpl)

                    #if (openpl > triggerprice):
                        #print('openpl > triggerprice = True')
                    #if (df['ST_Upperband'][previous_key]> 0) :
                        #print('Supertrend Line changed color = True')


                    profitloss = round((quantity * (limitprice-avgPrice)),2)


                    #sellEQUITY in TS
                    #sellEQUITY(symbol, quantity, limitprice, tradelog_filename)

                    #add to DB
                    addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H-%M-%S'), str(symbol), 'SELL_STOCK', limitprice, quantity)
                    addprofitlossOutputtoDB(profitloss)
                    deletePositionfromDB(symbol)

                    #add log to DB
                    addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELL_STOCK', limitprice, quantity, profitloss, bidPrice, askPrice,df)


            if (positionType == 'Short'):        
                  
                #(df['ST_Lowerband'][previous_key]> 0)  or 
                if ((askPrice >= stop_loss) or (askPrice <= profit_target)):

                    
                    #if (openpl_percent >= 0.1 or openpl_percent <= -0.30) :
                    #if (openpl_percent >= 0.1) :

                    print(df[['SYMBOL','TimeStamp','Close','Open','High','Low','K_slow','D_slow','ST_Upperband','ST_Lowerband']].tail(5))

                    print (datetime.now().strftime('%H:%M:%S'))
                    limitprice = round((askPrice+0.01),2)

                    print('Buy to Cover for ', symbol, 'at', limitprice, 'with p/l = $', openpl)

                    #if (openpl > triggerprice):
                        #print('openpl > triggerprice = True')
                    #if (df['ST_Lowerband'][previous_key]> 0) :
                        #print('Supertrend Line changed color = True')


                    profitloss = round((quantity * (avgPrice-limitprice)),2)

                    #buytocoverEQUITY in TS
                    #buytocoverEQUITY(symbol, quantity, limitprice, tradelog_filename)

                    #add to DB
                    addOrderOutputtoDB(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), str(symbol), 'BUYTOCOVER_STOCK', limitprice, quantity)
                    addprofitlossOutputtoDB(profitloss)
                    deletePositionfromDB(symbol)


                    #add log to DB
                    addOrderLogOutputtoDB(datetime_now, symbol, strategy,'BUYTOCOVER_STOCK', limitprice, quantity, profitloss, bidPrice, askPrice,df)

    #print("******************")
        
        
    except Exception as e:
        traceback.print_exc()
        print(e)
        pass


    
while True:
        
    now = datetime.now()
    today801=now.replace(hour=8,minute=0,second=0,microsecond=0)
    today1500=now.replace(hour=23,minute=0,second=0,microsecond=0)

    if (now > today801 and now < today1500 ) :
        #symbol_list = getStocks()
        #print(symbol_list)
        #getStrategyResult(symbol_list)
        
        #Set trade signals to True or False based on market movement
        sellstrategy = get_Sell_Strategy_Trade_Signals_fromDB()
        if (sellstrategy == True):
            
            # Time starts from here
            start = time.time()       

            #update option values in the DB - moved to separate
            #updatePositionValuesinDB()    

            positions = getDFFromPositionTable()
            #print(positions)


            if (len(positions) > 0):
                with concurrent.futures.ThreadPoolExecutor(max_workers=len(positions)) as executor:
                    #print(threading.currentThread().ident)            
                    executor.map(getStrategyResult,positions)


            # Total time calculated
            print(now, ' - The program takes ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(10)

        '''
        # close all positions on market close or certain time    
        marketclose=now.replace(hour=14,minute=59,second=0,microsecond=0)
        if (now > marketclose) :
            positions = getDFFromPositionTable()
            if (len(positions) > 0):
                with concurrent.futures.ThreadPoolExecutor(max_workers=len(positions)) as executor:
                    #print(threading.currentThread().ident)            
                    executor.map(closeAllPositionsBeforeMarketClose,positions)
        break       

        '''

    
        



2023-02-03 09:08:17.104065  - The program takes  0.04797005653381348 seconds.
**************************************************************
2023-02-03 09:08:27.218068  - The program takes  0.046872615814208984 seconds.
**************************************************************
2023-02-03 09:08:37.303979  - The program takes  0.04897022247314453 seconds.
**************************************************************
2023-02-03 09:08:47.390761  - The program takes  0.046874046325683594 seconds.
**************************************************************
2023-02-03 09:08:57.462788  - The program takes  0.031249284744262695 seconds.
**************************************************************
2023-02-03 09:09:07.549259  - The program takes  0.0421450138092041 seconds.
**************************************************************
2023-02-03 09:09:17.640673  - The program takes  0.046874046325683594 seconds.
**************************************************************
2023-02-03

Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_1248\1936675017.py", line 72, in getStrategyResult
    ema9 = df['ema9']
TypeError: list indices must be integers or slices, not str


2023-02-03 09:13:49.703004  - The program takes  0.5949430465698242 seconds.
**************************************************************
2023-02-03 09:14:00.350902  - The program takes  0.5686511993408203 seconds.
**************************************************************
2023-02-03 09:14:10.970182  - The program takes  0.6209461688995361 seconds.
**************************************************************
2023-02-03 09:14:21.644590  - The program takes  0.5183868408203125 seconds.
**************************************************************
2023-02-03 09:14:32.244251  - The program takes  0.752732515335083 seconds.
**************************************************************
2023-02-03 09:14:43.067463  - The program takes  0.9347236156463623 seconds.
**************************************************************
2023-02-03 09:14:54.056803  - The program takes  0.7149078845977783 seconds.
**************************************************************
2023-02-03 09:15:04.8

2023-02-03 09:23:53.527265  - The program takes  1.251668930053711 seconds.
**************************************************************
2023-02-03 09:24:04.813496  - The program takes  0.76432204246521 seconds.
**************************************************************
2023-02-03 09:24:15.624703  - The program takes  0.7111377716064453 seconds.
**************************************************************
2023-02-03 09:24:26.397139  - The program takes  1.6240220069885254 seconds.
**************************************************************
2023-02-03 09:24:38.070051  - The program takes  0.6889214515686035 seconds.
**************************************************************
2023-02-03 09:24:48.834255  - The program takes  0.6366932392120361 seconds.
**************************************************************
2023-02-03 09:24:59.504930  - The program takes  0.6850669384002686 seconds.
**************************************************************
2023-02-03 09:25:10.257

Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_1248\1936675017.py", line 72, in getStrategyResult
    ema9 = df['ema9']
TypeError: list indices must be integers or slices, not str


...

2023-02-03 10:47:12.875499  - The program takes  0.8948299884796143 seconds.
**************************************************************
2023-02-03 10:47:23.829731  - The program takes  0.7772777080535889 seconds.
**************************************************************
2023-02-03 10:47:34.672850  - The program takes  0.7762260437011719 seconds.
**************************************************************
2023-02-03 10:47:45.504255  - The program takes  0.7971255779266357 seconds.
**************************************************************
2023-02-03 10:47:56.368685  - The program takes  0.6898949146270752 seconds.
**************************************************************
2023-02-03 10:48:07.117995  - The program takes  0.7662386894226074 seconds.
**************************************************************
2023-02-03 10:48:17.960254  - The program takes  0.7641394138336182 seconds.
**************************************************************
2023-02-03 10:48:28.

2023-02-03 10:56:50.788777  - The program takes  0.8084051609039307 seconds.
**************************************************************
2023-02-03 10:57:01.638291  - The program takes  0.7757875919342041 seconds.
**************************************************************
2023-02-03 10:57:12.453494  - The program takes  0.8383402824401855 seconds.
**************************************************************
2023-02-03 10:57:23.327738  - The program takes  0.8990433216094971 seconds.
**************************************************************
2023-02-03 10:57:34.265433  - The program takes  0.9401357173919678 seconds.
**************************************************************
2023-02-03 10:57:45.254813  - The program takes  0.8068537712097168 seconds.
**************************************************************
2023-02-03 10:57:56.109685  - The program takes  0.857832670211792 seconds.
**************************************************************
2023-02-03 10:58:07.0

2023-02-03 11:03:21.148079  - The program takes  0.7060503959655762 seconds.
**************************************************************
2023-02-03 11:03:31.930648  - The program takes  0.6551945209503174 seconds.
**************************************************************
2023-02-03 11:03:42.643988  - The program takes  0.696134090423584 seconds.
**************************************************************
2023-02-03 11:03:53.384145  - The program takes  0.6775226593017578 seconds.
**************************************************************
2023-02-03 11:04:04.105593  - The program takes  0.7178406715393066 seconds.
**************************************************************
2023-02-03 11:04:14.891997  - The program takes  0.6593801975250244 seconds.
**************************************************************
2023-02-03 11:04:25.587297  - The program takes  0.8283181190490723 seconds.
**************************************************************
2023-02-03 11:04:36.4

2023-02-03 11:12:57.517374  - The program takes  0.7360625267028809 seconds.
**************************************************************
2023-02-03 11:13:08.310859  - The program takes  0.7395820617675781 seconds.
**************************************************************
2023-02-03 11:13:19.091683  - The program takes  0.7286124229431152 seconds.
**************************************************************
2023-02-03 11:13:29.866274  - The program takes  1.0362956523895264 seconds.
**************************************************************
2023-02-03 11:13:40.955152  - The program takes  0.7540683746337891 seconds.
**************************************************************
2023-02-03 11:13:51.775004  - The program takes  0.8584582805633545 seconds.
**************************************************************
2023-02-03 11:14:02.711777  - The program takes  0.8125226497650146 seconds.
**************************************************************
2023-02-03 11:14:13.

2023-02-03 11:23:41.814831  - The program takes  0.714360237121582 seconds.
**************************************************************
2023-02-03 11:23:52.568556  - The program takes  1.772082805633545 seconds.
**************************************************************
2023-02-03 11:24:04.414885  - The program takes  0.765631914138794 seconds.
**************************************************************
2023-02-03 11:24:15.227394  - The program takes  0.7080552577972412 seconds.
**************************************************************
2023-02-03 11:24:25.987588  - The program takes  0.7121102809906006 seconds.
**************************************************************
2023-02-03 11:24:36.737726  - The program takes  0.877469539642334 seconds.
**************************************************************
2023-02-03 11:24:47.665564  - The program takes  0.7150747776031494 seconds.
**************************************************************
2023-02-03 11:24:58.4486